<a href="https://colab.research.google.com/github/fabiobasson/Bi-Master/blob/main/geological_image_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import zipfile
import shutil
import numpy as np
import glob
import random
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
!rm -rf geological_similarity

**Importar arquivos de configuração do Kaglle**

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fabiobasson","key":"d71cbfdfd23379909173204eac74cf96"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

###  Download dos dados / descompactando os dados

In [33]:
!kaggle datasets download fabiobasson/geological-similarity
! unzip -qq  geological-similarity

geological-similarity.zip: Skipping, found more recently modified local copy (use --force to force download)


In [32]:
! rm -rf gneiss andesite

### Data Preprocessing

In [34]:
andesite_dir = glob.glob('andesite/*.jpg')
gneiss_dir = glob.glob('gneiss/*.jpg')
marble_dir = glob.glob('geological_similarity/marble/*.jpg');quartzite_dir = glob.glob('geological_similarity/quartzite/*.jpg')
rhyolite_dir = glob.glob('geological_similarity/rhyolite/*.jpg');schist_dir = glob.glob('geological_similarity/schist/*.jpg')
print(len(andesite_dir));print(len(gneiss_dir)); print(len(marble_dir)); print(len(quartzite_dir));  print(len(schist_dir))


1008
948
0
0
0


In [35]:
andesite_dir[0:4]

['andesite/9FUK8.jpg',
 'andesite/3VFJB.jpg',
 'andesite/7YPFT.jpg',
 'andesite/3PTNB.jpg']

In [ ]:
#geological_similarity = 'geological_similarity'
#subfolders = [f.path for f in os.scandir(geological_similarity) if f.is_dir()]


In [36]:
class_names = ['andesite', 'gneiss', 'marbler', 'quartzite', 'rhyolite',
               'schist']
class_names[0]

'andesite'

In [37]:
gneiss_df=[]
andesite_df=[]
for i in andesite_dir:
    andesite_df.append([i,class_names[0]])
for j in gneiss_dir:
   gneiss_df.append([j,class_names[1]])
df = andesite_df + gneiss_df
random.shuffle(df)
df

[['andesite/0M6H4.jpg', 'andesite'],
 ['gneiss/1NJM6.jpg', 'gneiss'],
 ['gneiss/0EUHA.jpg', 'gneiss'],
 ['gneiss/8IG8U.jpg', 'gneiss'],
 ['gneiss/8YMX3.jpg', 'gneiss'],
 ['andesite/1T895.jpg', 'andesite'],
 ['andesite/3BZ2V.jpg', 'andesite'],
 ['gneiss/7KU0I.jpg', 'gneiss'],
 ['andesite/8HKSE.jpg', 'andesite'],
 ['andesite/5ENX3.jpg', 'andesite'],
 ['gneiss/4E4H8.jpg', 'gneiss'],
 ['andesite/4AXA4.jpg', 'andesite'],
 ['gneiss/8PUXU.jpg', 'gneiss'],
 ['gneiss/0SA1U.jpg', 'gneiss'],
 ['andesite/5V8SI.jpg', 'andesite'],
 ['gneiss/2UJX4.jpg', 'gneiss'],
 ['andesite/8PDOC.jpg', 'andesite'],
 ['andesite/0KD1O.jpg', 'andesite'],
 ['andesite/4Y74U.jpg', 'andesite'],
 ['gneiss/0LONE.jpg', 'gneiss'],
 ['andesite/6CP9P.jpg', 'andesite'],
 ['andesite/4TOB3.jpg', 'andesite'],
 ['andesite/6HVI8.jpg', 'andesite'],
 ['andesite/7V21B.jpg', 'andesite'],
 ['gneiss/0AF29.jpg', 'gneiss'],
 ['gneiss/5LOWN.jpg', 'gneiss'],
 ['gneiss/6PJFW.jpg', 'gneiss'],
 ['gneiss/1JJ6Y.jpg', 'gneiss'],
 ['andesite/2CBMU.jp

In [38]:
data_df = pd.DataFrame(df,columns=['path','label'])
data_df

,path,label
0,andesite/0M6H4.jpg,andesite
1,gneiss/1NJM6.jpg,gneiss
2,gneiss/0EUHA.jpg,gneiss
3,gneiss/8IG8U.jpg,gneiss
4,gneiss/8YMX3.jpg,gneiss
...,...,...
1951,andesite/4E8TC.jpg,andesite
1952,gneiss/5PGTN.jpg,gneiss
1953,gneiss/5DUM7.jpg,gneiss
1954,gneiss/5MN3Q.jpg,gneiss


In [79]:
data_df.columns

RangeIndex(start=0, stop=18, step=1)